# Exercises

In this section we have two exercises:
1. Implement the polynomial kernel.
2. Implement the multiclass C-SVM.

## Polynomial kernel

You need to extend the ``build_kernel`` function and implement the polynomial kernel if the ``kernel_type`` is set to 'poly'. The equation that needs to be implemented:
\begin{equation}
K=(X^{T}*Y)^{d}.
\end{equation}

In [33]:
def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    elif kernel_type == 'poly':
        d = 2  
        kernel = kernel ** d  
    return kernel

## Implement a multiclass C-SVM

Use the classification method that we used in notebook 7.3 and IRIS dataset to build a multiclass C-SVM classifier. Most implementation is about a function that will return the proper data set that need to be used for the prediction. You need to implement:
- ``choose_set_for_label``
- ``get_labels_count``

In [34]:
def choose_set_for_label(data_set, label):
    from sklearn.datasets import load_iris
    import numpy as np
    
    iris = load_iris()
    
    train_data_set = data_set.copy()  
    
    train_labels = np.ones((len(data_set),))
    train_labels[labels != label] = -1.0
    
    test_data_set = test_data_set_global.copy()  
    test_labels = np.ones((len(test_data_set),))
    test_labels[test_labels_global != label] = -1.0
    return train_data_set, test_data_set, train_labels, test_labels

In [35]:
def get_labels_count(data_set):
    import numpy as np
    
    labels_count = len(np.unique(labels))
    return labels_count

In [36]:
def train(train_data_set, train_labels, kernel_type='linear', C=10, threshold=1e-5):
    kernel = build_kernel(train_data_set, kernel_type=kernel_type)

    P = train_labels * train_labels.transpose() * kernel
    q = -np.ones((objects_count, 1))
    G = np.concatenate((np.eye(objects_count), -np.eye(objects_count)))
    h = np.concatenate((C * np.ones((objects_count, 1)), np.zeros((objects_count, 1))))

    A = train_labels.reshape(1, objects_count)
    A = A.astype(float)
    b = 0.0

    sol = cvxopt.solvers.qp(cvxopt.matrix(P), cvxopt.matrix(q), cvxopt.matrix(G), cvxopt.matrix(h), cvxopt.matrix(A), cvxopt.matrix(b))

    lambdas = np.array(sol['x'])

    support_vectors_id = np.where(lambdas > threshold)[0]
    vector_number = len(support_vectors_id)
    support_vectors = train_data_set[support_vectors_id, :]

    lambdas = lambdas[support_vectors_id]
    targets = train_labels[support_vectors_id]

    b = np.sum(targets)
    for n in range(vector_number):
        b -= np.sum(lambdas * targets * np.reshape(kernel[support_vectors_id[n], support_vectors_id], (vector_number, 1)))
    b /= len(lambdas)

    return lambdas, support_vectors, support_vectors_id, b, targets, vector_number

def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    return kernel

def classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id):
    kernel = np.dot(test_data_set, support_vectors.T)
    sigma = 1.0
    #K = np.dot(test_data_set, support_vectors.T)
    #kernel = build_kernel(train_data_set, kernel_type='rbf')
    c = (1. / sigma * np.sum(test_data_set ** 2, axis=1) * np.ones((1, np.shape(test_data_set)[0]))).T
    c = np.dot(c, np.ones((1, np.shape(kernel)[1])))
    #aa = np.dot((np.diag(K)*np.ones((1,len(test_data_set)))).T[support_vectors_id], np.ones((1, np.shape(K)[0]))).T
    sv = (np.diag(np.dot(train_data_set, train_data_set.T))*np.ones((1,len(train_data_set)))).T[support_vectors_id]
    aa = np.dot(sv,np.ones((1,np.shape(kernel)[0]))).T
    kernel = kernel - 0.5 * c - 0.5 * aa
    kernel = np.exp(kernel / (2. * sigma ** 2))

    y = np.zeros((np.shape(test_data_set)[0], 1))
    for j in range(np.shape(test_data_set)[0]):
        for i in range(vector_number):
            y[j] += lambdas[i] * targets[i] * kernel[j, i]
        y[j] += b
    return np.sign(y)

In [37]:
import numpy as np
import cvxopt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

iris = load_iris()
data_set = iris.data
labels = iris.target

train_data_set_global, test_data_set_global, train_labels_global, test_labels_global = train_test_split(
    data_set, labels, test_size=0.2, random_state=42)

unique_labels_count = get_labels_count(data_set)  
print(f"Number of unique classes: {unique_labels_count}")

final_predictions = np.zeros(len(test_data_set_global))

original_labels = labels
labels = train_labels_global  

for class_label in range(unique_labels_count):
    print(f"Training classifier for class {class_label}")
    objects_count = len(train_data_set_global)
    
    train_data, test_data, train_bin_labels, test_bin_labels = choose_set_for_label(
        train_data_set_global, class_label)
    
    lambdas, support_vectors, support_vectors_id, b, targets, vector_number = train(
        train_data, train_bin_labels, kernel_type='rbf')
    
    predictions = classify_rbf(test_data_set_global, train_data, lambdas, targets, 
                             b, vector_number, support_vectors, support_vectors_id)
    
    if class_label == 0:
        confidence_scores = predictions.copy()
    else:
        confidence_scores = np.column_stack((confidence_scores, predictions))

labels = original_labels

final_predictions = np.argmax(confidence_scores, axis=1)

accuracy = accuracy_score(test_labels_global, final_predictions)
print(f"Multiclass SVM accuracy: {accuracy:.4f}")

Number of unique classes: 3
Training classifier for class 0
     pcost       dcost       gap    pres   dres
 0:  1.1634e+02 -1.8056e+03  4e+03  2e-01  2e-15
 1:  7.7529e+01 -1.6880e+02  3e+02  5e-03  2e-15
 2:  1.0115e+01 -2.2189e+01  3e+01  3e-15  3e-15
 3: -4.8661e-01 -4.9596e+00  4e+00  4e-16  1e-15
 4: -1.4222e+00 -2.4756e+00  1e+00  2e-16  5e-16
 5: -1.6628e+00 -2.1247e+00  5e-01  4e-16  2e-16
 6: -1.7800e+00 -1.9873e+00  2e-01  2e-16  1e-16
 7: -1.8258e+00 -1.9518e+00  1e-01  2e-16  1e-16
 8: -1.8609e+00 -1.8731e+00  1e-02  4e-16  2e-16
 9: -1.8660e+00 -1.8662e+00  2e-04  2e-16  1e-16
10: -1.8661e+00 -1.8661e+00  2e-06  2e-16  2e-16
Optimal solution found.
Training classifier for class 1
     pcost       dcost       gap    pres   dres
 0:  1.1473e+02 -2.4495e+03  5e+03  2e-01  2e-15
 1:  9.3901e+01 -2.2202e+02  4e+02  6e-03  2e-15
 2:  1.2918e+01 -2.5931e+01  4e+01  2e-05  3e-15
 3: -1.1759e-01 -5.5532e+00  5e+00  4e-16  2e-15
 4: -1.3443e+00 -2.3367e+00  1e+00  5e-16  5e-16
 5: 

In [38]:
iris = load_iris()
data_set = iris.data
labels = iris.target

train_data_set_global, test_data_set_global, train_labels_global, test_labels_global = train_test_split(
    data_set, labels, test_size=0.2, random_state=42)

original_labels = labels.copy()
labels = train_labels_global.copy()

unique_labels_count = get_labels_count(data_set)
print(f"Number of unique classes: {unique_labels_count}")

confidence_scores = None
for class_label in range(unique_labels_count):
    objects_count = len(train_data_set_global)
    print(f"Training classifier for class {class_label}")
    
    train_data, test_data, train_bin_labels, test_bin_labels = choose_set_for_label(
        train_data_set_global, class_label)
    
    lambdas, support_vectors, support_vectors_id, b, targets, vector_number = train(
        train_data, train_bin_labels, kernel_type='rbf')
    
    predictions = classify_rbf(test_data_set_global, train_data, lambdas, targets,
                             b, vector_number, support_vectors, support_vectors_id)
    
    if class_label == 0:
        confidence_scores = predictions.copy()
    else:
        confidence_scores = np.column_stack((confidence_scores, predictions))

labels = original_labels

final_predictions = np.argmax(confidence_scores, axis=1)

accuracy = accuracy_score(test_labels_global, final_predictions)
print(f"Multiclass SVM with RBF kernel accuracy: {accuracy:.4f}")

Number of unique classes: 3
Training classifier for class 0
     pcost       dcost       gap    pres   dres
 0:  1.1634e+02 -1.8056e+03  4e+03  2e-01  2e-15
 1:  7.7529e+01 -1.6880e+02  3e+02  5e-03  2e-15
 2:  1.0115e+01 -2.2189e+01  3e+01  3e-15  3e-15
 3: -4.8661e-01 -4.9596e+00  4e+00  4e-16  1e-15
 4: -1.4222e+00 -2.4756e+00  1e+00  2e-16  5e-16
 5: -1.6628e+00 -2.1247e+00  5e-01  4e-16  2e-16
 6: -1.7800e+00 -1.9873e+00  2e-01  2e-16  1e-16
 7: -1.8258e+00 -1.9518e+00  1e-01  2e-16  1e-16
 8: -1.8609e+00 -1.8731e+00  1e-02  4e-16  2e-16
 9: -1.8660e+00 -1.8662e+00  2e-04  2e-16  1e-16
10: -1.8661e+00 -1.8661e+00  2e-06  2e-16  2e-16
Optimal solution found.
Training classifier for class 1
     pcost       dcost       gap    pres   dres
 0:  1.1473e+02 -2.4495e+03  5e+03  2e-01  2e-15
 1:  9.3901e+01 -2.2202e+02  4e+02  6e-03  2e-15
 2:  1.2918e+01 -2.5931e+01  4e+01  2e-05  3e-15
 3: -1.1759e-01 -5.5532e+00  5e+00  4e-16  2e-15
 4: -1.3443e+00 -2.3367e+00  1e+00  5e-16  5e-16
 5: 